In [ ]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
import numpy as np

from vn1.score import compute_competition_score

In [ ]:
base_path = Path("../mlruns/594158418212352111/6cdc635627e540728f8f89e5cdf3f700/artifacts/")
run_id = "6cdc635627e540728f8f89e5cdf3f700"

In [ ]:
preds = {}
for fold in range(4):
    preds[fold] = pd.read_pickle(base_path / f"predictions_{run_id}_fold_{fold}.p")

In [ ]:
def compute_cv_score_multiplier(preds, multiplier):
    scores = {}
    for fold,(y_pred,y_test) in preds.items():
        y_pred_corrected = y_pred * multiplier
        err = y_pred_corrected - y_test
        score = compute_competition_score(y_pred_corrected,y_test)
        scores[fold] = score
    
    return float(sum(scores.values()) / len(scores))

In [ ]:
compute_cv_score_multiplier(preds, multiplier=1)

In [ ]:
compute_cv_score_multiplier(preds, multiplier=0.99)

In [ ]:
multiplier_scores = []
multipliers = np.linspace(0.995,1,21)
for multiplier in multipliers:
    score = compute_cv_score_multiplier(preds, multiplier=multiplier)
    multiplier_scores.append(score)
    #print(multiplier, score)

multiplier_df = pd.DataFrame({"multiplier": multipliers, "score": multiplier_scores})

fig = go.Figure([go.Scatter(x=multiplier_df.multiplier, y=multiplier_df.score)])
fig.update_layout(width=600)
fig.show()

In [ ]:
# seems like 0.9955 is the minimum